In [142]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [143]:
import os
import sys

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('../')
os.getcwd()

'/home/lehl/development/QueryOptDeepDive'

In [144]:
import os
import re
import datetime
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [145]:
id2aliasname = {
    0: 'start', 1: 'chn', 2: 'ci', 3: 'cn', 4: 'ct', 5: 'mc', 6: 'rt', 7: 't', 8: 'k', 9: 'lt',
    10: 'mk', 11: 'ml', 12: 'it1', 13: 'it2', 14: 'mi', 15: 'mi_idx', 16: 'it', 17: 'kt', 18: 'miidx', 19: 'at',
    20: 'an', 21: 'n', 22: 'cc', 23: 'cct1', 24: 'cct2', 25: 'it3', 26: 'pi', 27: 't1', 28: 't2', 29: 'cn1',
    30: 'cn2', 31: 'kt1', 32: 'kt2', 33: 'mc1', 34: 'mc2', 35: 'mi_idx1', 36: 'mi_idx2', 37: 'an1', 38: 'n1', 39: 'a1'
}

aliasname2id = {
    'kt1': 31, 'chn': 1, 'cn1': 29, 'mi_idx2': 36, 'cct1': 23, 'n': 21, 'a1': 39, 'kt2': 32, 'miidx': 18, 'it': 16,
    'mi_idx1': 35, 'kt': 17, 'lt': 9, 'ci': 2, 't': 7, 'k': 8, 'start': 0, 'ml': 11, 'ct': 4, 't2': 28, 'rt': 6,
    'it2': 13, 'an1': 37, 'at': 19, 'mc2': 34, 'pi': 26, 'mc': 5, 'mi_idx': 15, 'n1': 38, 'cn2': 30, 'mi': 14,
    'it1': 12, 'cc': 22, 'cct2': 24, 'an': 20, 'mk': 10, 'cn': 3, 'it3': 25, 't1': 27, 'mc1': 33
}

len(id2aliasname.keys()), len(aliasname2id.keys())

(40, 40)

In [146]:
tmp = set()

for query_file in all_queries:
    
    with open(os.path.join('balsa', 'queries', 'join-order-benchmark', query_file), 'r') as f:
        query = ''.join(f.readlines())
    
    query = query.split('FROM')[1].split('WHERE')[0]
    conds = query.split(',')
    
    for cond in conds:
        cond = cond.strip()
        table_name, table_alias = cond.split(' AS ')
        
        if table_alias not in aliasname2id.keys():
            tmp.add(cond)
            
    if '15' in query_file:
        print(query_file)
        print(query)

tmp

15b.sql
 aka_title AS at,
     company_name AS cn,
     company_type AS ct,
     info_type AS it1,
     keyword AS k,
     movie_companies AS mc,
     movie_info AS mi,
     movie_keyword AS mk,
     title AS t

15d.sql
 aka_title AS at,
     company_name AS cn,
     company_type AS ct,
     info_type AS it1,
     keyword AS k,
     movie_companies AS mc,
     movie_info AS mi,
     movie_keyword AS mk,
     title AS t

15a.sql
 aka_title AS aka_t,
     company_name AS cn,
     company_type AS ct,
     info_type AS it1,
     keyword AS k,
     movie_companies AS mc,
     movie_info AS mi,
     movie_keyword AS mk,
     title AS t

15c.sql
 aka_title AS at,
     company_name AS cn,
     company_type AS ct,
     info_type AS it1,
     keyword AS k,
     movie_companies AS mc,
     movie_info AS mi,
     movie_keyword AS mk,
     title AS t



{'aka_title AS aka_t'}

In [147]:
def query_file_to_ident(file_name):
    ident = file_name.split('.sql')[0]
    return f"{ident[:-1].zfill(2)}{ident[-1]}"

QUERY_TIMEOUT = 2 * 3 * 60 * 1000

In [148]:
"""
TEST_QUERIES = dict()

for folder in os.listdir('bao/queries'):
    if not folder.startswith('job__'):
        continue
        
    key = folder.split('job__')[1]
    
    queries = os.listdir(os.path.join('bao', 'queries', folder, 'test'))
    queries = sorted([query_file_to_ident(x) for x in queries])

    TEST_QUERIES[key] = queries
    
for k in sorted(TEST_QUERIES.keys()):
    print(k)
    print(TEST_QUERIES[k])
    print('---' * 30)
"""    

TEST_QUERIES = {
    'base_query_split_1': ['02a', '02b', '02c', '02d', '07a', '07b', '07c', '15a', '15b', '15c', '15d', '24a', '24b', '25a', '25b', '25c', '31a', '31b', '31c'],
    'base_query_split_2': ['13a', '13b', '13c', '13d', '15a', '15b', '15c', '15d', '20a', '20b', '20c', '26a', '26b', '26c', '29a', '29b', '29c', '30a', '30b', '30c', '33a', '33b', '33c'],
    'base_query_split_3': ['01a', '01b', '01c', '01d', '05a', '05b', '05c', '12a', '12b', '12c', '17a', '17b', '17c', '17d', '17e', '17f', '22a', '22b', '22c', '22d', '27a', '27b', '27c', '28a', '28b', '28c'],
    'leave_one_out_split_1': ['01c', '02a', '03b', '04a', '05a', '06b', '07c', '08c', '09c', '10b', '11b', '12c', '13b', '14a', '15b', '16c', '17c', '18b', '19a', '20c', '21c', '22b', '23b', '24a', '25a', '26c', '27c', '28a', '29b', '30a', '31b', '32b', '33c'],
    'leave_one_out_split_2': ['01d', '02d', '03a', '04b', '05c', '06d', '07a', '08c', '09c', '10a', '11a', '12a', '13d', '14b', '15b', '16a', '17f', '18a', '19d', '20a', '21b', '22c', '23b', '24b', '25a', '26a', '27b', '28c', '29a', '30b', '31a', '32b', '33b'],
    'leave_one_out_split_3': ['01c', '02d', '03b', '04a', '05c', '06d', '07b', '08a', '09a', '10c', '11d', '12a', '13a', '14b', '15a', '16d', '17b', '18b', '19d', '20b', '21a', '22a', '23b', '24a', '25b', '26a', '27a', '28b', '29c', '30a', '31a', '32a', '33c'],
    'random_split_1': ['01c', '02c', '04b', '04c', '05c', '06a', '06c', '06e', '08b', '08c', '09c', '11d', '15a', '17b', '17e', '18b', '20a', '21a', '25c', '28b', '32b', '33a'],
    'random_split_2': ['01a', '04c', '05c', '06c', '06d', '07b', '08c', '10a', '11a', '11d', '13c', '13d', '15d', '16a', '17b', '19a', '20a', '22b', '25b', '29b', '31a', '32b'],
    'random_split_3': ['02a', '03b', '06d', '09b', '10b', '11b', '11c', '13c', '13d', '16b', '18c', '19c', '21c', '22a', '22d', '26a', '26b', '27c', '28a', '28c', '30a', '33c'],
}

In [149]:
all_queries = os.listdir('balsa/queries/join-order-benchmark')
all_queries = [q for q in all_queries if q.endswith('sql')]
print(all_queries)

['22b.sql', '33a.sql', '31c.sql', '14c.sql', '16d.sql', '17d.sql', '17c.sql', '19b.sql', '1d.sql', '10b.sql', '5b.sql', '25b.sql', '30c.sql', '2a.sql', '20b.sql', '22d.sql', '21a.sql', '24a.sql', '1a.sql', '21c.sql', '12b.sql', '21b.sql', '7a.sql', '27b.sql', '23a.sql', '18b.sql', '11b.sql', '8b.sql', '10c.sql', '12a.sql', '32b.sql', '7b.sql', '6b.sql', '8a.sql', '26c.sql', '9d.sql', '22a.sql', '1b.sql', '3b.sql', '6d.sql', '25a.sql', '33c.sql', '9c.sql', '6e.sql', '19c.sql', '6a.sql', '2c.sql', '12c.sql', '3c.sql', '27c.sql', '8c.sql', '11a.sql', '29a.sql', '1c.sql', '9a.sql', '26a.sql', '2b.sql', '18c.sql', '5c.sql', '25c.sql', '33b.sql', '13b.sql', '11d.sql', '4a.sql', '6f.sql', '28a.sql', '18a.sql', '22c.sql', '24b.sql', '15b.sql', '20a.sql', '16c.sql', '32a.sql', '4c.sql', '27a.sql', '19a.sql', '23c.sql', '20c.sql', '8d.sql', '10a.sql', '3a.sql', '17a.sql', '6c.sql', '17b.sql', '31b.sql', '28c.sql', '31a.sql', '29b.sql', '13a.sql', '29c.sql', '5a.sql', '11c.sql', '9b.sql', '28b.sq

In [154]:
def build_workload(queries):
    tmp = []

    for q in queries:
        query_ident = query_file_to_ident(q)

        with open(os.path.join('balsa', 'queries', 'join-order-benchmark', q), 'r') as f:
            lines = f.readlines()

        sql = ''.join(lines)
        sql = sql.split(';')[0] + ';'
        
        if 'aka_t' in sql:
            sql = sql.replace('aka_title AS aka_t', 'aka_title AS at')
            sql = sql.replace('aka_t.', 'at.')
            
        tmp.append([sql, query_ident, [-1, False]])
        
    return tmp

In [157]:
for key in TEST_QUERIES.keys():
    print(f'WORKLOAD: {key}')
    #os.remove(f"{base_path}__train.json")
    #os.remove(f"{base_path}__test.json")
    
    test_queries = []
    for q in TEST_QUERIES[key]:
        ident = f"{str(int(q[:2]))}{q[2]}.sql"
        test_queries.append(ident)
    
    train_queries = []
    for q in sorted(all_queries):
        if q not in test_queries:
            train_queries.append(q)
    
    base_path = f"hybrid_qo/workload/JOB__{key}"
    
    train_wl = build_workload(train_queries)
    with open(f"{base_path}__train.json", 'w') as f:
        json.dump(train_wl, f)
    
    test_wl = build_workload(test_queries)
    with open(f"{base_path}__test.json", 'w') as f:
        json.dump(test_wl, f)
    


WORKLOAD: base_query_split_1
WORKLOAD: base_query_split_2
WORKLOAD: base_query_split_3
WORKLOAD: leave_one_out_split_1
WORKLOAD: leave_one_out_split_2
WORKLOAD: leave_one_out_split_3
WORKLOAD: random_split_1
WORKLOAD: random_split_2
WORKLOAD: random_split_3


In [158]:
sorted(os.listdir('hybrid_qo/workload'))

['JOB__base_query_split_1__test.json',
 'JOB__base_query_split_1__train.json',
 'JOB__base_query_split_2__test.json',
 'JOB__base_query_split_2__train.json',
 'JOB__base_query_split_3__test.json',
 'JOB__base_query_split_3__train.json',
 'JOB__leave_one_out_split_1__test.json',
 'JOB__leave_one_out_split_1__train.json',
 'JOB__leave_one_out_split_2__test.json',
 'JOB__leave_one_out_split_2__train.json',
 'JOB__leave_one_out_split_3__test.json',
 'JOB__leave_one_out_split_3__train.json',
 'JOB__random_split_1__test.json',
 'JOB__random_split_1__train.json',
 'JOB__random_split_2__test.json',
 'JOB__random_split_2__train.json',
 'JOB__random_split_3__test.json',
 'JOB__random_split_3__train.json',
 'JOB_dynamic.json',
 'JOB_static.json']